In [1]:
import os
import json
import pandas as pd
from PIL import Image
from tqdm import *
import numpy as np
import pickle
from shutil import copyfile
from mmdet.apis import init_detector, inference_detector, show_result
import glob

import torch.utils.data as data

from PIL import Image
import os
import os.path

import cv2
import sys
from copy import deepcopy
import torchvision
import torch
from mmdet.ops.nms import nms_cpu, nms
from skimage.feature import match_template
from skimage.color import rgb2gray
import numpy as np
import time

from psina_track_now_staff import *

In [91]:
IMAGES_DATA_PATH = "/home/mml6/IceChallenge/test/"
PREDICTIONS_PATH = "Final_model/full_skolkovo_final_stage0_842.pkl"
PRED_THRESHOLD = 0.55
IOU_NMS_THRESHOLD = 0.05
MIN_BBOX_SQUARE = 100
TRACKING_MIN_IOU = 0.001

In [3]:
with open(PREDICTIONS_PATH, 'rb') as fin:  
    detector_predictions = filter_all_predictions(
        pickle.load(fin),
        PRED_THRESHOLD,
        IOU_NMS_THRESHOLD,
        MIN_BBOX_SQUARE)

In [4]:
file_names = []

for cur_img in glob.glob(IMAGES_DATA_PATH + "**", recursive=True):
    if not ".jpg" in cur_img:
        continue
    cur_img = '/'.join(cur_img.split('/')[-2:])
    file_names.append(cur_img)

file_names = np.array(file_names)
video_seq = np.argsort(file_names)[::-1]
selected_filenames = file_names[video_seq]

In [5]:
with open('all_classes.txt') as f:
    all_pos_classes = f.read().split()

convert_class = lambda x: '.'.join(str(x).split('.')[:2])

all_pos_classes = [convert_class(sign) for sign in all_pos_classes]

valid_classes = sorted(
    ['2.1',
     '2.4',
     '3.1',
     '3.24',
     '3.27',
     '4.1',
     '4.2',
     '5.19',
     '5.20',
     '8.22'])

In [92]:
def match_bboxes(start_frame_bboxes, finish_frame_bboxes):
    # Returns list of pairs (start_bbox, finish_bbox)
    # where each bbox is a pair (bbox, class_id)
    
    start_to_finish_iou = []
    for i, start_bbox in enumerate(start_frame_bboxes):
        for j, finish_bbox in enumerate(finish_frame_bboxes):
            score = iou(start_bbox[0], finish_bbox[0])
            if start_bbox[1] != finish_bbox[1]:
                score = 0
            
            start_to_finish_iou.append((score, i, j))
    
    start_to_finish_iou.sort(key=lambda x: x[0], reverse=True)
    
    matched_pairs = []
    used_start_bboxes = set()
    used_finish_bboxes = set()
    
    for score, i, j in start_to_finish_iou:
        if score < TRACKING_MIN_IOU:
            break
    
        if i in used_start_bboxes or j in used_finish_bboxes:
            continue
        
        start_bbox = start_frame_bboxes[i][:-1]
        finish_bbox = finish_frame_bboxes[j][:-1]
        matched_pairs.append((start_bbox, finish_bbox))
        
        used_start_bboxes.add(i)
        used_finish_bboxes.add(j)

    return matched_pairs

In [102]:
def run_tracking(sequence_tracking):
    result = []

    start_frame = sequence_tracking[0]
    finish_frame = sequence_tracking[-1]

    interpolate_frames = sequence_tracking[1:-1]

    matched_bboxes = match_bboxes(start_frame[0], finish_frame[0])

    for frame_num, (_, image) in enumerate(interpolate_frames):
        frame_bboxes = []
        for start_bbox, finish_bbox in matched_bboxes:
            class_id = start_bbox[1]
            start_bbox = start_bbox[0]
            finish_bbox = finish_bbox[0]
            
            bbox_diff = [
                finish_bbox[0] - start_bbox[0],
                finish_bbox[1] - start_bbox[1],
                finish_bbox[2] - start_bbox[2],
                finish_bbox[3] - start_bbox[3]]
            
            new_bbox = start_bbox[:]
            for i in range(4):
                new_bbox[i] += bbox_diff[i] / (len(interpolate_frames) + 1) * (frame_num + 1)
            
            frame_bboxes.append((new_bbox, class_id, 1))
        
        result.append(frame_bboxes)

    result.append(finish_frame[0])
    return result

In [111]:
DETECTOR_FREQUENCY = 3

dataset = ImageFilelist(IMAGES_DATA_PATH, selected_filenames)
loader = torch.utils.data.DataLoader(dataset,
                                     shuffle=False,
                                     num_workers=12)
loader = iter(loader)

final_boxes = []
start_time = time.time()

current_sequence_tracking = []
current_sequence_index = 0

for ind in range(len(selected_filenames))[:]:
    if ind == 0 or selected_filenames[ind].split('/')[0] != selected_filenames[ind-1].split('/')[0]:
        # NEW SEQUENCE starts!!!
        if len(current_sequence_tracking) > 0:
            for boxes, _ in current_sequence_tracking[1:]:
                final_boxes.append(boxes)
        
        current_sequence_tracking = []
        current_sequence_index = 0

    cur_img = next(loader)[0][0].data.numpy()
    cur_img_gray = cv2.cvtColor(cur_img, cv2.COLOR_BGR2GRAY)

    frame_predictions = detector_predictions[ind]
    
    frame_final_boxes = []    
    if (not current_sequence_tracking) or ((len(current_sequence_tracking) + 1) % DETECTOR_FREQUENCY == 0):
        frame_final_boxes = []
        for prediction in frame_predictions:
            bbox = prediction[:4]
            class_id = int(prediction[4])
            frame_final_boxes.append((bbox, class_id, 0))
    
    current_sequence_tracking.append((frame_final_boxes, cur_img_gray))
    
    if len(current_sequence_tracking) == DETECTOR_FREQUENCY:
        if current_sequence_index == 0:
            final_boxes.append(current_sequence_tracking[0][0])
            current_sequence_index += 1
        for boxes in run_tracking(current_sequence_tracking):
            final_boxes.append(boxes)
        
        current_sequence_tracking = [current_sequence_tracking[-1]]

    current_time = int(time.time() - start_time)
    time_per_iter = (time.time() - start_time) / (ind + 1)
    eta_time = int((len(selected_filenames) - ind) * time_per_iter)
    print("\rIndex: {}. Time: {} s. ETA: {} s".format(ind, current_time, eta_time), end="")

if len(current_sequence_tracking) > 0:
    for boxes, _ in current_sequence_tracking[1:]:
        final_boxes.append(boxes)

Index: 14999. Time: 92 s. ETA: 0 s

In [112]:
with open('tracking_check.tsv', 'w') as f:
    f.write('\t'.join(['frame', 'xtl', 'ytl', 'xbr', 'ybr', 'class']) + '\n')
    
    for ind in range(len(selected_filenames)):
        img_name = selected_filenames[ind]
        img_name = img_name.replace('.jpg', '')
        for bbox, class_id, _ in final_boxes[ind]:
            class_id = int(class_id)
            class_name = all_pos_classes[class_id]
            if class_name not in valid_classes:
                continue
            bbox = hw_to_min_max(bbox)
            bbox = list(map(str, bbox))
            f.write('\t'.join([img_name, *bbox, class_name]) + '\n')

In [113]:
!./score/target/release/icevision-score /media/mml6/HDD/Ice/annotations/final ~/IceChallenge/tracking_check.tsv

Total score:	693.312
Total penalty:	164.000
Score 2.1:	33.760
Score 2.4:	78.517
Score 3.1:	38.315
Score 3.24:	-6.891
Score 3.27:	53.388
Score 4.1:	46.942
Score 4.2:	53.963
Score 5.19:	228.120
Score 5.20:	137.709
Score 8.22:	29.489
Penalty 2.1:	16.000
Penalty 2.4:	38.000
Penalty 3.1:	14.000
Penalty 3.24:	36.000
Penalty 3.27:	8.000
Penalty 4.1:	12.000
Penalty 4.2:	4.000
Penalty 5.19:	36.000
Penalty 5.20:	0.000
Penalty 8.22:	0.000


In [ ]:
# REFERENCE SCORE

# Total score:	819.665
# Total penalty:	142.000
# Score 2.1:	43.085
# Score 2.4:	102.378
# Score 3.1:	57.288
# Score 3.24:	-2.041
# Score 3.27:	54.694
# Score 4.1:	58.172
# Score 4.2:	54.263
# Score 5.19:	283.166
# Score 5.20:	135.837
# Score 8.22:	32.821
# Penalty 2.1:	14.000
# Penalty 2.4:	28.000
# Penalty 3.1:	8.000
# Penalty 3.24:	36.000
# Penalty 3.27:	8.000
# Penalty 4.1:	10.000
# Penalty 4.2:	4.000
# Penalty 5.19:	24.000
# Penalty 5.20:	10.000
# Penalty 8.22:	0.000

In [11]:
ind = 0
for cur_name, frame_final_predictions in zip(selected_filenames[:1000], final_boxes):
    image = cv2.imread(os.path.join(IMAGES_DATA_PATH, cur_name))
    for bbox, class_id, source in frame_final_predictions:
        class_name = all_pos_classes[int(class_id)]
        if class_name not in valid_classes:
            continue
        bbox = hw_to_min_max(bbox)
        
        if source == 0: # detector
            cv2.rectangle(image, (int(bbox[0]), int(bbox[1])),
                          (int(bbox[2]), int(bbox[3])),
                          (0, 255, 0), 2)
        elif source == 1: # tracking
            cv2.rectangle(image, (int(bbox[0]), int(bbox[1])),
                          (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
    
        cv2.putText(image,
                        class_name,# + ' ' + str(cur_prob)[:4],
                        (int(bbox[0]), int(bbox[1])),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.5,
                        (0, 255, 0),
                        thickness=3,
                        lineType=cv2.LINE_AA) 
    
    cv2.imwrite('VisualTracking/' + cur_name.split('/')[1], image)
    ind += 1

KeyboardInterrupt: 